In [55]:
import numpy as np

def generate_lower_triangular_dependency_matrix(size):
    # 下三角行列をランダム生成（対角成分は0にする）
    A = np.tril(np.random.randint(0, 2, (size, size)), k=-1).astype(float)
    
    # 各ノードが必ず1つの依存関係を持つように調整
    for i in range(size - 1):  # 最後のノード以外をチェック
        if not A[i, :].any():  # 出次数が0のノードを確認
            # ランダムに依存関係を作る
            j = np.random.randint(i + 1, size)  # 自分より後のノードから選択
            A[j, i] = 1.0
    
    return A

# 行列サイズを指定
size = 6
dependency_matrix = generate_lower_triangular_dependency_matrix(size)
print(dependency_matrix)

# 加えて、列で見たときに、残りの下の行と、1が一つもない列の数を数えて、行の数が列の数と等しければフラグを立たすと言う制約を入れる。

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 0. 0.]]


データ生成で必要な条件  
- 初期状態からの遷移(1~N/2)くらい？（上位N/2行でok）
- 何かしらのノードのつながっている
- 三角行列（一方向のみ）

In [126]:
import numpy as np

def generate_lower_triangular_dependency_matrix(size):
    # 下三角行列をランダム生成（対角成分は0にする）
    A = np.tril(np.random.randint(0, 2, (size, size)), k=-1).astype(float)

    # 初期状態からの遷移をいくつ作るか
    init = np.random.randint(1, int(size / 2))
    print(init)
    for i in range(init):
        A[i+1] = 0
        A[i+1, 0] = 1
    for j in range(init, size-1):
        A[j+1, 0] = 0
    
    # 各列いずれかに1が格納されるように調整（下三角行列内で）
    for i in range(size):
        if not A[:, i].any():  # i列がすべて0の場合
            # i+1以降の行で0要素目が1でないものをフィルタリング
            candidates = [row for row in range(i + 1, size) if A[row, 0] != 1]
            if candidates:  # 候補が存在する場合のみ処理
                j = np.random.choice(candidates)  # ランダムに行を選択
                A[j, i] = 1.0
                
    # 各ノードが必ず1つの依存関係を持つように調整
    for i in range(size - 1):  # 最後のノード以外をチェック
        if not A[i, :].any():  # 出次数が0のノードを確認
            if i > 1:  # i = 0 の場合をスキップ
                # ランダムに依存関係を作る
                j = np.random.randint(1, i)  # 1からi-1までの数字から選択
                A[i, j] = 1.0

    return A


def refine_dependency_matrix(A):
    size = A.shape[0]
    for i in range(size):
        for j in range(i + 1, size):
            # A[j][1:] が A[i][1:] を包含しているかを確認
            if all(A[j, 1:] >= A[i, 1:]) and np.any(A[i, 1:] > 0):
                # A[j][i] を 1 にし、それ以外を 0 に設定
                A[j, :] = A[j, :] - A[i, :]
                A[j, i] = 1
    return A

# 行列サイズを指定
size = 6
# 初期の依存行列を生成
dependency_matrix = generate_lower_triangular_dependency_matrix(size)
print("Original Dependency Matrix:")
print(dependency_matrix)

# 包含関係に基づいて修正
refined_matrix = refine_dependency_matrix(dependency_matrix)
print("Refined Dependency Matrix:")
print(refined_matrix)


1
Original Dependency Matrix:
[[0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0.]]
Refined Dependency Matrix:
[[0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]
